In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 97 kB 5.9 MB/s 
     |████████████████████████████████| 180 kB 44.4 MB/s 
     |████████████████████████████████| 139 kB 45.8 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from collections import namedtuple
import numpy as np
import os
from sklearn.utils import shuffle

Dataset = namedtuple("Dataset", ["x", "r", "fi"])

In [ ]:
sweep_config = {

    'method': 'bayes',
    'metric': {
      'name': 'val_mae',
      'goal': 'minimize'   
    },
    'parameters': {
        'target': {
            'values': ['fi']
        },
        'epochs': {
           'values': [100]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'layer_1_size': {
                'values': [5040, 5760, 6480]
        },
        'layer_2_size': {
                'values': [5760, 6480, 7200]
        },
        'layer_3_size': {
                'values': [1440, 2160, 2880, 3600]
        },
        'learning_rate': {
            'distribution': 'uniform',
            'max': 0.002,
            'min': 2e-06
        },
        'optimizer': {
            'distribution': 'categorical',
            'values': ['sgd', 'rmsprop']
        },
        'activation1': {
            'distribution': 'categorical',
            'values': ['tanh', 'relu']
        },
        'activation2': {
            'distribution': 'categorical',
            'values': ['relu', 'tanh']
        },
        'activation3': {
            'distribution': 'categorical',
            'values': ['relu']
        },
        'activation4': {
            'distribution': 'categorical',
            'values': ['relu']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="artem-starkov", project="flatfasetgen")

Create sweep with ID: rpqvtdqm
Sweep URL: https://wandb.ai/artem-starkov/flatfasetgen/sweeps/rpqvtdqm


In [ ]:
def read(data_dir, split):
    filename = split + ".npz"
    data = np.load(os.path.join(data_dir, filename))

    return Dataset(x=data["x"], r=data["r"], fi=data["fi"])

In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 50,
        'batch_size': 16,
        'learning_rate': 1e-6,
        'activation1': 'relu',
        'activation2': 'relu',
        'activation3': 'relu',
        'activation4': 'relu',
        'optimizer': 'sgd',
        'layer_1_size': 4096,
        'layer_2_size': 4096,
        'layer_3_size': 4096,
        'm': 720,
        'h': 40,
        'l' : 20,
        'rMin': 50,
        'rMax': 1000,
        'gMax': 100,
        'target': 'fi'
    }

    run = wandb.init(project="flatfasetgen", job_type="training_angle_w/o_d.out", config=config_defaults)
    processed_data = wandb.Artifact("Clear_datasets", type="dataset")
    raw_data_artifact = run.use_artifact('Clear_datasets:v10') 
    raw_dataset = raw_data_artifact.download()
    dataset = read(raw_dataset, 'full_set')
    run.log_artifact(processed_data)
    X = np.array(dataset.x)
    y = np.array(dataset.fi)
    X, y = shuffle(X, y)
    X_train = X[:200000]
    y_train = y[:200000]
    X_test = X[200000:]
    y_test = y[200000:]
    #X_train, X_test, y_train, y_test = train_test_split(dataset.x, dataset.fi, test_size=0.2)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    model = Sequential()
    model.add(Dense(config.m, input_shape=(config.m,)))
    model.add(Activation(config.activation1))

    model.add(Dense(config.layer_1_size))
    model.add(Activation(config.activation2))

    model.add(Dense(config.layer_2_size))
    model.add(Activation(config.activation3))

    model.add(Dense(config.layer_3_size))
    model.add(Activation(config.activation4))


    model.add(Dense(1))

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = SGD(lr=config.learning_rate, decay=1e-5, nesterov=True)
    elif config.optimizer=='rmsprop':
      optimizer = RMSprop(lr=config.learning_rate, decay=1e-5)

    model.compile(loss='mae', optimizer = optimizer, metrics=['mae', 'mape'])

    model_artifact = wandb.Artifact(
            "angle_compiled_model", type="model",
            description=f"bayes, 100k, best params from skratch, target: {config.target}",
            metadata=dict(config))
    model.save("angle_compiled_models")
    model_artifact.add_dir("angle_compiled_models")
    run.log_artifact(model_artifact)

    model.fit(X_train, y_train, batch_size=config.batch_size,
              epochs=config.epochs, validation_data=(X_test, y_test),
              callbacks=[WandbCallback(validation_data=(X_test, y_test)),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    wandb.log({"R2" : r2_score(y_test, model.predict(X_test))})
    wandb.log({"RMSE": mean_squared_error(y_test, model.predict(X_test), squared=False)})
    trained_model_artifact = wandb.Artifact(
            "angle_trained_model", type="model",
            description="",
            metadata=dict(config))
    
    model.save('angle_trained_models')
    trained_model_artifact.add_dir('angle_trained_models')
    run.log_artifact(trained_model_artifact)
    run.finish()

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 73ucpl54 with config:
wandb: 	activation1: relu
wandb: 	activation2: relu
wandb: 	activation3: relu
wandb: 	activation4: relu
wandb: 	batch_size: 64
wandb: 	epochs: 100
wandb: 	layer_1_size: 5040
wandb: 	layer_2_size: 5760
wandb: 	layer_3_size: 2880
wandb: 	learning_rate: 0.0002911956808269649
wandb: 	optimizer: sgd
wandb: 	target: fi
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: Downloading large artifact Clear_datasets:v10, 1398.63MB. 1 files... Done. 0:0:0
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
wandb: Adding directory to artifact (./angle_compiled_models)... Done. 1.5s


Epoch 1/100
3125/3125 [==============================] - 77s 23ms/step - loss: 0.3161 - mae: 0.3161 - mape: 18.4046 - val_loss: 0.1643 - val_mae: 0.1643 - val_mape: 9.7385
Epoch 2/100
3125/3125 [==============================] - 72s 23ms/step - loss: 0.1344 - mae: 0.1344 - mape: 8.1208 - val_loss: 0.1126 - val_mae: 0.1126 - val_mape: 6.9766
Epoch 3/100
3125/3125 [==============================] - 72s 23ms/step - loss: 0.1009 - mae: 0.1009 - mape: 6.3242 - val_loss: 0.0905 - val_mae: 0.0905 - val_mape: 5.7757
Epoch 4/100
3125/3125 [==============================] - 72s 23ms/step - loss: 0.0843 - mae: 0.0843 - mape: 5.4262 - val_loss: 0.0782 - val_mae: 0.0782 - val_mape: 5.1172
Epoch 5/100
3125/3125 [==============================] - 72s 23ms/step - loss: 0.0740 - mae: 0.0740 - mape: 4.8714 - val_loss: 0.0693 - val_mae: 0.0693 - val_mape: 4.6254
Epoch 6/100
3125/3125 [==============================] - 72s 23ms/step - loss: 0.0667 - mae: 0.0667 - mape: 4.4674 - val_loss: 0.0631 - val_mae:

wandb: Adding directory to artifact (./angle_trained_models)... Done. 1.7s


R2,▁
RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▅▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mape,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
R2,0.98642
RMSE,0.05403


...

wandb: Adding directory to artifact (./angle_trained_models)... Done. 2.9s


R2,▁
RMSE,▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▆▄▁▅▁▁▁▂▄▂▁▁▃▁▅▁▂
val_mae,█▂▆▄▁▅▁▁▁▂▄▂▁▁▃▁▅▁▂
val_mape,█▃▁▂▅▇▄▃▄▃▂▃▅▄▂▃▁▅▂
R2,-0.0
RMSE,0.46407


wandb: Agent Starting Run: ed0gtrdu with config:
wandb: 	activation1: tanh
wandb: 	activation2: relu
wandb: 	activation3: relu
wandb: 	activation4: relu
wandb: 	batch_size: 128
wandb: 	epochs: 100
wandb: 	layer_1_size: 6480
wandb: 	layer_2_size: 7200
wandb: 	layer_3_size: 2880
wandb: 	learning_rate: 0.0001738477742368927
wandb: 	optimizer: rmsprop
wandb: 	target: fi
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: Downloading large artifact Clear_datasets:v10, 1398.63MB. 1 files... Done. 0:0:0
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
wandb: Adding directory to artifact (./angle_compiled_models)... Done. 2.3s


Epoch 1/100
1563/1563 [==============================] - 133s 84ms/step - loss: 0.1593 - mae: 0.1593 - mape: 10.1519 - val_loss: 0.0755 - val_mae: 0.0755 - val_mape: 4.8817
Epoch 2/100
1563/1563 [==============================] - 131s 84ms/step - loss: 0.0748 - mae: 0.0748 - mape: 4.6585 - val_loss: 0.0486 - val_mae: 0.0486 - val_mape: 2.9495
Epoch 3/100
1563/1563 [==============================] - 130s 83ms/step - loss: 0.0567 - mae: 0.0567 - mape: 3.5010 - val_loss: 0.0495 - val_mae: 0.0495 - val_mape: 2.9110
Epoch 4/100
1563/1563 [==============================] - 129s 82ms/step - loss: 0.0466 - mae: 0.0466 - mape: 2.8571 - val_loss: 0.0495 - val_mae: 0.0495 - val_mape: 2.9284
Epoch 5/100
1563/1563 [==============================] - 128s 82ms/step - loss: 0.0399 - mae: 0.0399 - mape: 2.4201 - val_loss: 0.0353 - val_mae: 0.0353 - val_mape: 2.2382
Epoch 6/100
1563/1563 [==============================] - 129s 82ms/step - loss: 0.0347 - mae: 0.0347 - mape: 2.0892 - val_loss: 0.0362 - va

In [ ]:
while True:
  pass